In [1]:
%pip install torch
%pip install tf-keras

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
from transformers import OpenAIGPTTokenizer, OpenAIGPTModel, GPT2Tokenizer, GPT2Model
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
# 1) load tokenizers & models
ogpt_tok = OpenAIGPTTokenizer.from_pretrained("openai-gpt")
ogpt_mod = OpenAIGPTModel.from_pretrained("openai-gpt")
g2_tok   = GPT2Tokenizer.from_pretrained("gpt2")
g2_mod   = GPT2Model.from_pretrained("gpt2")

ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.


In [4]:
# 2) tokenize the single word
word = "chopped"
ogpt_ids = ogpt_tok(word, return_tensors="pt")     
g2_ids   = g2_tok(word,   return_tensors="pt")
print(ogpt_ids)
print(g2_ids)

{'input_ids': tensor([[17163]]), 'attention_mask': tensor([[1]])}
{'input_ids': tensor([[6679, 1496]]), 'attention_mask': tensor([[1, 1]])}


In [5]:
# 3) forward pass to get last hidden states
with torch.no_grad():
    ogpt_hid = ogpt_mod(**ogpt_ids).last_hidden_state  # (1, seq_len1, dim1)
    g2_hid   = g2_mod(**g2_ids).last_hidden_state      # (1, seq_len2, dim2)

In [6]:
# 4) mean‐pool across the subtoken dimension
def mean_pool(hid, mask):
    mask = mask.unsqueeze(-1)           # (1, seq_len, 1)
    summed = (hid * mask).sum(1)        # (1, dim)
    counts = mask.sum(1)                # (1, 1)
    return summed / counts

ogpt_emb = mean_pool(ogpt_hid, ogpt_ids.attention_mask)  
g2_emb   = mean_pool(g2_hid,   g2_ids.attention_mask)   

In [7]:
# 5) compute cosine similarity
sim = cosine_similarity(
    ogpt_emb.cpu().numpy(),
    g2_emb.cpu().numpy()
)[0][0]

# the higher the cosine similarity, the more similar the embeddings (ie. higher the match)
print(f"Cosine similarity between OpenAI-GPT and GPT-2 “{word}” embeddings: {1 - sim:.4f}")

Cosine similarity between OpenAI-GPT and GPT-2 “chopped” embeddings: 0.9573
